In [5]:
cc.VerificationHandler.close_browser()

In [4]:
VerificationHandler.close_browser()

In [12]:
import numpy as np
import pandas as pd

In [1]:
% run contactsScraper.py

Done
Done
KEYS
Done
Done
RECORDS COLLECTED
DATAFRAMES READY


In [2]:
% run contactChecker.py

Local Contact Checker Ready


In [5]:
ContactSheetOutput.set_output(contactKeys)
VerificationHandler.set_orgRecords(dm.OrgSession(orgRecords))
VerificationHandler.set_contactRecords(cr)
print('Local Contact Checker Ready')

RECORD OUTPUT READY
Local Contact Checker Ready


In [6]:
orgsForToday = ['National Association for Multi-Ethnicity In Communications (NAMIC)',
                'Association for Women in Science',
                'Brain Injury Association of America',
                'American Society of Home Inspectors',
                'NAADAC, the Association for Addiction Professionals',
                'American Public Transportation Association',
                'Indiana Soybean Alliance',
                'Associated Builders and Contractors (ABC)',
                'National Association of Social Workers',
                'American Marketing Association (AMA)']

In [7]:
org  = orgsForToday[0]

vh = MotherSetVerifier(org)

In [8]:
VerificationHandler.orgRecords.orgSessionStatusCheck()

['Sat Oct 21, 2017  11:10:09', '11.344', 's', 'Good']

In [9]:
ContactSheetOutput.currentRow

243

In [10]:
vh.write_contact_pointers()

In [11]:
vh.pointers

In [13]:
## Identify Grandmother elements
pointers = vh.verifiedPointers
gmElements = []
gmMatrix = []

for i in range(len(pointers)):
    igmElements = []
    for j in range(i):
        ## Check to see if the Any Mother element is a Big Momma or "Bertha" Element
        if pointers[i].get_mother_element() is pointers[j].get_mother_element():
            gm = pointers[i].get_mother_element()
        else:
            gm = pointers[i].common_parent(pointers[j])
        # Append Match to Grand Mother Matrix
        igmElements.append(gm)
        
        # Check to see if this is a new grand mother element,
        # if so append to the gmElements list of unique grandmother elements 
        if gm not in gmElements:
            gmElements.append(gm)
            
    # Append Matrix Row
    gmMatrix.append(igmElements)
    
grandMotherMatrix = np.matrix(gmMatrix)

In [14]:
noGm = len(gmElements)

In [15]:
noGm

1

In [21]:
## if 1 grandmother element test set it and test it
gm = gmElements[0]

In [29]:
def distinct_gm(gm, pts):
    if len(pts) == 0:
        return True
    if gm is pts[0].get_mother_element():
        return False
    else:
        return distinct_gm(gm, pts[1:])

In [31]:
## check to see if Grandmother is Distinct
distinct = distinct_gm(gm, pointers)

In [32]:
distinct

True

In [25]:
pointers[1].get_mother_element()

<div class="team-member-right"><h3 class="team-member-name">S. Monique Wells</h3><h4 class="team-member-position">Director of Finance and Administration</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">Monique Wells is the Director of Finance and Administration for the National Association for Multi-ethnicity in Communications’ (NAMIC). Prior to that she was their Manager of Operations, a position she assumed in August, 2009. Monique is no stranger to NAMIC and has been a member since 1991. She served as a consultant to NAMIC for a number of special projects, prior to joining the staff. As Director of Finance and Administration, she oversees the administrative and financial operations of the national office. An accomplished professional with a diverse background in sales and project management, Monique previously served as an Energy Conservation Analyst with the Los Angeles Unified School District where she oversaw the utility management for sch

In [34]:
## Tag tree with pattern from nathan to tom
def tag_nathans(pt):
    return parent_cycle_up(pt.get_mother_element(), pt.nathan.parent, 'nathan', 0)

def tag_toms(pt):
    return parent_cycle_up(pt.get_mother_element(), pt.tom.parent, 'tom', 0)

def parent_cycle_up(motherElement, element, atr, num):
    ## for the bertha case we stop at the element that is right befor the mother elemenet
    if element.parent is motherElement:
        element[atr] = num
        element['sib'] = atr
    else:
        element[atr] = num
        return(motherElement, element.parent, atr, num + 1)

In [35]:
## For Each pointer
# Tag Toms and Nathans
vp = pointers[0]
tag_nathans(vp)
tag_toms(vp)

In [40]:
vp.tom.parent

<h4 class="team-member-position" sib="tom" tom="0">Senior Manager, Membership and Publications</h4>

In [41]:
## Rocket Extraction: Are you ready to launch?
vp.nathan.parent['class']

['team-member-name']

In [42]:
vp.tom.parent['class']

['team-member-position']

In [44]:
vp.get_mother_element()['class']

['team-member-right']

In [46]:
## Tom Starts
tomStarts = gm.find_all(class_=vp.tom.parent['class'])

In [48]:
## Nathan Starts
nathanStarts = gm.find_all(class_=vp.nathan.parent['class'])

In [52]:
## Mother Starts
motherStarts = gm.find_all(class_=vp.get_mother_element()['class'])

In [58]:
vp.tom.parent.previous_sibling

<h3 class="team-member-name" nathan="0" sib="nathan">Susan Waldman</h3>

In [54]:
vp.tom.parent.previous_sibling

'Susan Waldman'

In [55]:
vp.nathan.parent.next_sibling

<h4 class="team-member-position" sib="tom" tom="0">Senior Manager, Membership and Publications</h4>

In [62]:
# Getting to a nathan with a tom start
n = 5
tomStarts[n].previous_sibling.string

'Darryl Jackson'

In [65]:
# Getting to a tom with a nathan Start
n = 0
nathanStarts[n].next_sibling.string

'President and CEO'

In [114]:
tom_rocket = lambda startNode: startNode.next_sibling.string


In [115]:
n = 7
tom_rocket(nathanStarts[n])

'Administrative Assistant'

In [116]:
nathan_rocket = lambda startNode: startNode.previous_sibling.string

In [117]:
n = 7
nathan_rocket(tomStarts[n])

"Davana O'Brien"

In [118]:
nathan_missile = lambda startNode: startNode.contents[0].string

In [119]:
n = 7
nathan_missile(motherStarts[n])

"Davana O'Brien"

In [120]:
tom_missile = lambda startNode: startNode.contents[0].next_sibling.string

In [121]:
n = 7
tom_missile(motherStarts[n])

'Administrative Assistant'

In [96]:
motherStarts[0].contents[0].string

'Eglon E. Simons'

In [36]:
vp.nathan.parent


<h3 class="team-member-name" nathan="0" sib="nathan">Susan Waldman</h3>

In [19]:
vp.tom.parent

<h4 class="team-member-position">Senior Manager, Membership and Publications</h4>

In [20]:
vp.mary

<div class="team-member-right"><h3 class="team-member-name">Susan Waldman</h3><h4 class="team-member-position">Senior Manager, Membership and Publications</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">As the Senior Manager of Membership and Publications, Susan Waldman oversees the day-to-day facilitation of NAMIC’s membership operations, as well as market research to drive growth. She is also responsible for the production of NAMIC’s industry-wide publication, Diversity Digest. She launched her career in market research as a media planner on such accounts as Loews Hotels and Omnipoint Communications at Hill, Holliday Altschiller and McCann Direct advertising agencies.  There she was introduced to the gamut of advertising research resources as she planned and purchased the media for various clients.  After a few years, Susan moved over to the magazine industry. Susan has served as the Marketing Research Manager at Vibe, Spin and The Source mag